In [7]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import numpy as np
import torch.nn.functional as F

Image.MAX_IMAGE_PIXELS = 432785250

# Load and preprocess the original and noisy images
original_image_path = 'C:\\Satellite-Image-Denoising-master\\Data\\Train\\Clean\\class_1\\1_0.jpeg'
noisy_image_path = 'C:\\Satellite-Image-Denoising-master\\Data\\Train\\Noisy\\class_1\\1_0.jpeg'

# Define the denoising autoencoder model
class DenoisingAutoencoder(nn.Module):
    def __init__(self):
        super(DenoisingAutoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=4),
            nn.Conv2d(32, 1, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Load the pre-trained model
model = DenoisingAutoencoder()
model.load_state_dict(torch.load('denoising_model_2.pth'))
# model.eval()

# Define data transformation for inference
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((1024, 1024)),
    transforms.ToTensor(),
])

# Load and preprocess a test image
image_path = 'denoised_image1.jpeg'
image_arr = Image.open(image_path)
image_arr = np.array(image_arr)

height, width = image_arr.shape

patch_width, patch_height = 1024, 1024
counter = 0

new_image_arr = image_arr.copy()

for hg in range(0, height, patch_height):
    for wd in range(0, width, patch_width):
        cropped_im_arr = image_arr[hg:hg+patch_height, wd:wd+patch_width]
        cropped_im = Image.fromarray(cropped_im_arr.astype(np.uint8))

        input_tensor = transform(cropped_im).unsqueeze(0)
        output_patch = model(input_tensor)

        output_image = transforms.ToPILImage()(output_patch.squeeze(0))
        arr = np.array(output_image)

        try:
            new_image_arr[hg:hg+patch_height, wd:wd+patch_width] = arr
        except ValueError:
            remaining_height = min(patch_height, height - hg)
            remaining_width = min(patch_width, width - wd)
            
            new_image_arr[hg:hg+patch_height, wd:wd+patch_width] = np.resize(arr, (remaining_height, remaining_width))

# Convert output image to 8-bit integer mode (grayscale)
output_image = Image.fromarray(new_image_arr).convert('L')

# Save the denoised image as JPEG
output_image.save('denoised_image13.jpg')

C:\Users\USER\AppData\Local\Temp\ipykernel_1488\1895921580.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('denoising_model_2.pth'))


In [8]:
import numpy as np
import math
# Calculate PSNR function
def calculate_psnr(image1, image2):
    mse = np.mean((image1 - image2) ** 2)
    if mse == 0:  # No noise
        return float('inf')
    max_pixel = 255.0
    psnr = 20 * math.log10(max_pixel / math.sqrt(mse))
    return psnr

# Load the original and denoised images
original_image = Image.open(image_path)
original_image_arr = np.array(original_image)

# Load and preprocess the clean (original) and noisy images
clean_image_path = 'C:\\Satellite-Image-Denoising-master\\Data\\Train\\Clean\\class_1\\1_0.jpeg'
clean_image = Image.open(clean_image_path)
# Convert images to numpy arrays
clean_image_arr = np.array(clean_image)

denoised_image = Image.open('denoised_image13.jpg')
denoised_image_arr = np.array(denoised_image)

# Calculate PSNR between the clean (original) and noisy image (before denoising)
# Calculate PSNR between the clean (original) and noisy image (before denoising)
psnr = calculate_psnr(clean_image_arr, denoised_image_arr)
print(f"PSNR after Denoising: {psnr:.2f} dB")



PSNR after Denoising: 28.21 dB
